In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, make_scorer
from torch.utils.data import DataLoader, TensorDataset
import scipy.stats as stats
from torchsummary import summary
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.base import BaseEstimator
from torchmetrics import MeanSquaredError, R2Score

## Load Data

In [2]:
df = pd.read_csv('./leap-atmospheric-physics-ai-climsim/train.csv', sep=',', nrows=2000000)

## Feature Selection

In [3]:
input_df = df.iloc[:, 1:557]
output_df = df.iloc[:, 557:]

In [4]:
# check which columns have very small variance
low_variance_cols = np.where(np.std(input_df, axis=0) < 1e-6)[0]
print(len(low_variance_cols))
print(low_variance_cols)

235
[ 60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 180 181 182 183 184 185 186
 187 188 189 190 191 192 193 194 195 196 376 377 378 379 380 381 382 388
 389 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410
 411 412 413 414 415 416 417 418 419 420 421 422 423 424 425 426 427 428
 429 430 431 432 433 434 435 436 437 438 439 440 441 442 443 444 445 446
 447 448 449 450 451 452 453 454 455 456 457 458 459 460 461 462 463 464
 465 466 467 468 469 470 471 472 473 474 475 476 477 478 479 480 481 482
 483 484 485 486 487 488 489 490 491 492 493 494 495 496 497 498 499 500
 501 502 503 504 505 506 507 508 509 510 511 512 513 514 515 516 517 518
 519 520 521 522 523 524 525 526 527 528 529 530 531 532 533 534 535 536
 537 538 539 540 541 542 543 544 545 546 547 548 549 550 551 552 553 554
 555]


In [5]:
# check if any of small variance features have correlation with outputs
correlated_cols = []

for col_index in low_variance_cols:
    correlation_check = 0
    for output_col in output_df.columns:
        input_col = input_df.columns[col_index]
        corr_coef = input_df[input_col].corr(output_df[output_col])
        if -0.1 <= corr_coef <= 0.1 or np.isnan(corr_coef):
            continue
        else:
            correlation_check = 1
            break
    if correlation_check == 1:
        correlated_cols.append(col_index)

print(correlated_cols)
print(len(correlated_cols))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[63, 64, 65, 66, 70, 71, 72, 73, 74, 75, 76, 77, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 183, 184, 188, 189, 190, 191, 192, 193, 194, 195, 196, 376, 377, 378, 379, 380, 381, 382, 388, 389, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522]
153


In [6]:
# create list of features with no correlation
set1 = set(low_variance_cols)
set2 = set(correlated_cols)

no_corr_or_var = list(set1 - set2)
print(len(no_corr_or_var))
print(no_corr_or_var)

82
[523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 60, 61, 62, 67, 68, 69, 120, 121, 122, 123, 180, 181, 182, 185, 186, 187, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495]


In [7]:
# drop low variance with no correlation columns
input_df_clean = input_df.drop(input_df.columns[no_corr_or_var], axis=1)
input_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Columns: 474 entries, state_t_0 to pbuf_N2O_26
dtypes: float64(474)
memory usage: 7.1 GB


In [8]:
# check normal distribution, use MinMaxScaler if most features are not normally distributed
p_threshold = 0.05
not_norm_num = 0
for col in input_df_clean.iloc[:, 1:].columns:
    stat, p = stats.normaltest(df[col])
    if p <= p_threshold:
        not_norm_num += 1

print(f'Number of input features not normally distributed = {not_norm_num} out of {len(input_df_clean.columns)}')

Number of input features not normally distributed = 473 out of 474
